In [66]:
import yaml
import os
from pprint import pprint
import re

from prompt import Prompt

In [2]:
config_file = "./config/config.yaml"
output_path = "../benchmark_dir"
agent_name = "ChatGPT_1" # Available: ChatGPT_1 (4o-mini), ChatGPT_2 (4o), Llaama_3_1, Gemma_2, Mistral
model_name = "gpt-4o-mini" # Available: gpt-4o-mini, gpt-4o, llama-3.1-70b-versatile, gemma2-9b-it, mixtral-8x7b-32768

In [3]:
def set_agent_configuration(configuration_file_path, agent_name):
    with open(configuration_file_path, mode="r") as file:
        config = yaml.load(file, Loader=yaml.Loader)

    agent_config = config["Agent"]

    if (agent_name == "ChatGPT_1") or (agent_name == "ChatGPT_2"):
        agent_config[agent_name]["api_keys"] = [os.environ["OPENAI_API_KEY"]]
    else:
        agent_config[agent_name]["api_keys"] = [os.environ["GROQ_API_KEY"]]
    return agent_config

In [4]:
agent_config = set_agent_configuration(configuration_file_path=config_file, agent_name=agent_name)

In [8]:
prompt = Prompt(
    agent_configuration=agent_config,
    agent_name=agent_name,
    model_name=model_name,
    few_shot_count=1
)

In [15]:
input_path = prompt.get_filepaths()["input"][0]
input_functions = prompt.get_input_functions(input_path)
input_documentation = prompt.extract_input_function_docs(input_path)

In [20]:
pprint(input_functions)

('- Write(range="Sheet2!A1", value="Customers")\n'
 '- Write(range="Sheet2!A1", value="Customers")\n'
 '- CreateSheet(sheetName="Sheet2")\n'
 '- Write(range="Sheet2!A1", value="Customers")\n'
 '- AutoFill(source="Sheet1!A2:E2", destination="Sheet2!A2:E11")\n'
 '- AutoFill(source="Sheet1!A2:E2", destination="Sheet2!A2:E11")\n'
 '- Write(range="Sheet2!A2", value="=Sheet1!A2")\n')


In [21]:
pprint(input_documentation)

['Write(range: str, value: str)\n'
 'Args explanation:\n'
 'range (string): The range to write the value into. Here, the range must be a '
 'single cell.\n'
 'value (string): The value to be written. The string in value also can be an '
 'Excel formula. A string starting with "=" will be treated as a formula. When '
 'using address indexing from another sheet in a formula, if the worksheet '
 'name contains spaces, its name must be enclosed in single quotes (e.g. '
 "'Pivot Table'!A1). Please use correct absolute and relative references, "
 'e.g., $A$1, $A1, A$1, A1.\n'
 '\n'
 'Usage example:\n'
 '# Example 1: Write headers for the new column.\n'
 'Write("Sheet2!A1", "Date")\n'
 'Write("Sheet2!B1", "Sales")\n'
 '# Example 2: Write the formula into the cell.\n'
 'Write("Sheet1!A2", "=VLOOKUP(\'Retail Price\'!C2, \'Retail Price\'!A:B, 2, '
 'FALSE)") # Must single quote the sheet name if it contains spaces.\n'
 'Write("Sheet1!A3", "=SUM(Sheet2!$A$1:$A$20)") # Absolute address.\n'
 '# Exa

In [22]:
input_path

'..\\benchmark_dir\\refined_responses\\10_EntireShippingCosts.yaml'

In [23]:
pprint(prompt.get_correct_summarization(prompt.get_filepaths()["response"][0]))

('- Step 1. Write the headers in Sheet2.\n'
 '- Step 1. Create the headers in Sheet2.\n'
 '- Step 1. Create Sheet2.\n'
 "- Step 1. Create a new sheet named 'Sheet2'.\n"
 '- Step 2. Write the headers in Sheet2.\n'
 '- Step 3. Autofill the first 10 rows of data from Sheet1 to Sheet2.\n'
 '- Step 3. Write the first 10 rows of data from Sheet1 to Sheet2 manually.\n'
 '- Step 3. Copy the first 10 rows of data from Sheet1 to Sheet2.\n'
 '- Step 3. Write the first 10 rows of data from Sheet1 to Sheet2 manually.\n')


In [29]:
path = "..\\benchmark_dir\\10_EntireShippingCosts\\10_EntireShippingCosts_log.yaml"
with open(path, mode="r") as file:
    log_file = yaml.load(file, Loader=yaml.Loader)


In [41]:
# refined_response = log_file["Success Response"][0]["refined response"]
# list(dict.fromkeys([item for sublist in refined_response for item in sublist]))

In [64]:
def extract_intermediate_responses(log_file):
    intermediate_response = log_file["Success Response"][0]["intermediate response"]
    intermediate_response = [sub_step[:sub_step.find("\nAction API: ")] for sub_step in intermediate_response]
    intermediate_response_dict = {"intermediate response": intermediate_response}
    return intermediate_response_dict

In [65]:
extract_intermediate_responses(log_file)

{'intermediate response': ['Step 1. Write the headers in Sheet2.',
  'Step 1. Create the headers in Sheet2.',
  'Step 1. Create Sheet2.',
  "Step 1. Create a new sheet named 'Sheet2'.",
  'Step 2. Write the headers in Sheet2.',
  'Step 3. Autofill the first 10 rows of data from Sheet1 to Sheet2.',
  'Step 3. Write the first 10 rows of data from Sheet1 to Sheet2 manually.',
  'Step 3. Copy the first 10 rows of data from Sheet1 to Sheet2.',
  'Step 3. Write the first 10 rows of data from Sheet1 to Sheet2 manually.']}

In [68]:
# def extract_refined_responses(log_file):
intermediate_response = log_file["Success Response"][0]["intermediate response"]
refined_response = [re.findall(r'(?<=@)([A-Z].*?\))(?=@|\n|$)', sub_step) for sub_step in intermediate_response]


9